# Installation

In [ ]:
# pip install -r requirements.txt

: 

# Tool usage test

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Add console handler only if not already present
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

Persona Extractor

In [3]:
from src.tools.persona_extractor import get_persona_extractor_tool

tool1 = get_persona_extractor_tool('user_41', 'Lifestyle Optimization')

result1 = tool1.invoke({"sentence": "I also like to eat pasta for dinner."})
print(result1)

Device set to use mps:0
INFO - SQLitePersonaDB initialized
INFO - [EXTRACTED] relation='favorite_food', object='pasta', topic='Skill', task='Lifestyle Optimization'
INFO - Persona fact inserted for user user_41, task Lifestyle Optimization
INFO - [SAVED TO DB] for username='user_41'


The persona has extracted. Just inform the user.


Community-based Recommender:

In [1]:
from src.tools.community_recommender import create_community_recommender_tool

tool2 = create_community_recommender_tool(username="user_41", task="Content Consumption")

result2 = tool2.invoke({"sentence": "Can you provide community recommendations?"})
print(result2)

INFO:persona_util:SQLitePersonaDB initialized
INFO:src.tools.community_recommender:[TOOL INVOKED] Community recommender triggered with input: Can you provide community recommendations?


[TOOL RESULT]
Type: Recommendation:
Community suggestions for Content Consumption (from other users):
- Book:
  . reading graphic novels: liked by 5 users
  . reading science fiction: liked by 5 users
  . reading cookbooks: liked by 4 users
  . reading mysteries: liked by 4 users
- Game:
  . card games: liked by 6 users
  . strategy games: liked by 6 users
  . arcade games: liked by 5 users
  . role-playing games: liked by 5 users
- Movie:
  . watching animated movies: liked by 6 users
  . watching fantasy movies: liked by 6 users
  . watching historical dramas: liked by 6 users
  . watching adventure movies: liked by 5 users
- Music:
  . hip hop music: liked by 6 users
  . indie music: liked by 6 users
  . reggae music: liked by 6 users
  . blues music: liked by 5 users


# Agent Testing

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Add console handler
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [3]:
from dotenv import load_dotenv
load_dotenv() 

True

In [4]:
from src.utils.persona_util import SQLitePersonaDB
db = SQLitePersonaDB()

USERNAME = "user_41"

db.create_user(USERNAME, 'Sali A', 28, 'male', 'user')

USER = db.get_user(USERNAME)

task_list = [f"{task['name']}" for task in db.get_all_tasks()]

TASK = task_list[0]
print(TASK)

USER_PERSONA = db.get_user_persona_summary_by_task('user_01', TASK)
USER_PERSONA = USER_PERSONA.replace('user_01', USERNAME)

USER_PERSONA

INFO - SQLitePersonaDB initialized
INFO - User created: user_41


Content Consumption


'User user_41 preferences for Content Consumption: | Book: likes reading novels | Game: plays video games | Movie: likes watching sci-fi | Music: likes classical music'

## OpenAI Agent

In [5]:
from src.agents.PersoAgent import PersoAgent

agent = PersoAgent(
    user = USER,
    task = TASK,
    prev_personas = USER_PERSONA
)

INFO - Initializing PersoAgent for user: {'username': 'user_41', 'full_name': 'Sali A', 'age': 28, 'gender': 'male', 'role': 'user'} with task: Content Consumption
Device set to use cuda:0
INFO - SQLitePersonaDB initialized
INFO - SQLitePersonaDB initialized


In [6]:
# Simple invocation - just pass the task
response = agent.handle_task("I also enjoy horror movies.")

print(response)



> Entering new AgentExecutor chain...


INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "response": "It's great to know that you enjoy horror movies!",
    "reason": "User expressing his/her preferences",
    "used_tool": "PersonaExtractor"
}

> Finished chain.
{
    "response": "It's great to know that you enjoy horror movies!",
    "reason": "User expressing his/her preferences",
    "used_tool": "PersonaExtractor"
}


In [7]:
response = agent.handle_task("Can you provide community recommendations?")

print(response)



> Entering new AgentExecutor chain...


INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO - [TOOL INVOKED] Community recommender triggered with input: Can you provide community recommendations?



Invoking: `CommunityRecommender` with `{'sentence': 'Can you provide community recommendations?'}`


[TOOL RESULT]
Type: Recommendation:
Community suggestions for Content Consumption (from other users):
- Book:
  . reading graphic novels: liked by 5 users
  . reading science fiction: liked by 5 users
  . reading cookbooks: liked by 4 users
- Game:
  . card games: liked by 6 users
  . strategy games: liked by 6 users
  . arcade games: liked by 5 users
- Movie:
  . watching animated movies: liked by 6 users
  . watching fantasy movies: liked by 6 users
  . watching historical dramas: liked by 6 users
- Music:
  . hip hop music: liked by 6 users
  . indie music: liked by 6 users
  . reggae music: liked by 6 users

INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "response": "Here are some community recommendations based on what other users like:",
    "reason": "User requested community recommendations",
    "used_tool": "CommunityRecommender"
}

> Finished chain.
{
    "response": "Here are some community recommendations based on what other users like:",
    "reason": "User requested community recommendations",
    "used_tool": "CommunityRecommender"
}


In [8]:
# Simple invocation - just pass the task
response = agent.handle_task("So, which movies should I watch tonight?")

print(response)



> Entering new AgentExecutor chain...


INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "response": "You can consider watching 'Inception' or 'The Shawshank Redemption' for a thrilling movie night!",
    "reason": "General movie recommendation based on popular choices",
    "used_tool": "None"
}

> Finished chain.
{
    "response": "You can consider watching 'Inception' or 'The Shawshank Redemption' for a thrilling movie night!",
    "reason": "General movie recommendation based on popular choices",
    "used_tool": "None"
}


# Labeling Assistatn

In [ ]:
# Set memory limits before importing transformers
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import torch
from transformers import GPT2Tokenizer, AutoModelForCausalLM
import json
import re

class LabelingAssistant:
    def __init__(self):
        # Use 75% of GPU memory (9GB out of 12GB)
        torch.cuda.set_per_process_memory_fraction(0.75)
        torch.cuda.empty_cache()
        
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = GPT2Tokenizer.from_pretrained("./Tokenizers/Phi-4-mini-instruct")
        
        self.model = AutoModelForCausalLM.from_pretrained(
            "./LLMs/Phi-4-mini-instruct", 
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token
    
    def get_alignment_score(self, user_persona: str, classification_description: str) -> dict:
        prompt = f"""<|user|>
Analyze alignment between user persona and classification. Return JSON with score (0.0-1.0) and reasoning.

User Persona: {user_persona}
Classification: {classification_description}

Response format: {{"score": 0.85, "reasoning": "explanation"}}
<|end|>
<|assistant|>"""

        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                use_cache=False
            )
        
        response = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        
        del outputs, inputs
        torch.cuda.empty_cache()
        
        try:
            json_match = re.search(r'\{[^}]*"score"[^}]*\}', response)
            if json_match:
                return json.loads(json_match.group())
        except:
            pass
        
        score_match = re.search(r'([0-9.]+)', response)
        score = float(score_match.group(1)) if score_match else 0.5
        return {"score": min(1.0, score), "reasoning": response.strip()}
    
    def reset_memory(self):
        torch.cuda.set_per_process_memory_fraction(1.0)
        torch.cuda.empty_cache()

# Usage
assistant = LabelingAssistant()

user_persona = "Content Consumption: | Book: likes reading novels | Game: plays video games | Movie: likes watching sci-fi | Music: likes classical music"
classification = "Classify users based on their passion for reading books and literature."

result = assistant.get_alignment_score(user_persona, classification)
print(f"Score: {result['score']}")
print(f"Reasoning: {result['reasoning']}")

assistant.reset_memory()


c:\Users\Admin\Desktop\persoagent_backend\.venv\Lib\site-packages\accelerate\utils\modeling.py:808: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Score: 0.5
Reasoning: The user persona shows a strong interest in reading novels, which aligns well with the classification's focus on reading books and literature. However, the user also enjoys other forms of media such as video games, sci-fi movies, and classical music, which are not directly related to the classification's focus on reading. Therefore, while there is a significant overlap, the user's diverse interests prevent a perfect score.


In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Add console handler
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [2]:
from src.utils.persona_util import SQLitePersonaDB

db = SQLitePersonaDB()

res = db.get_random_candidate_usernames(classification_task_name="Content Consumption", count=5)
res

INFO - SQLitePersonaDB initialized
INFO - Found 5 candidate users for classification task 'Content Consumption'


['user_36', 'user_19', 'user_02', 'user_06', 'user_34']

In [3]:
from src.tools.labeling_assistant import LabelingAssistant

assistant = LabelingAssistant()
pool_size = 5
clf_task_name = "Camping Enthusiast"
result = assistant.get_alignment_score(pool_size, clf_task_name)

c:\Users\Admin\Desktop\persoagent_backend\.venv\Lib\site-packages\accelerate\utils\modeling.py:808: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

INFO - SQLitePersonaDB initialized
INFO - Found 5 candidate users for classification task 'Camping Enthusiast'
INFO - Found 5 candidates for task 'Camping Enthusiast'
INFO - Persona Summary for user_31: 
User user_31 preferences: || Career Development - Career: prefers_path academia | Education: studied_at Columbia | Skill: has_skill writing || Content Consumption - Book: likes reading mysteries | Game: plays role-playing games | Movie: likes watching adventure movies | Music: likes blues music || Lifestyle Optimization - Exercise: has_hobby climbing | Mental Health: focuses_on reading self-help | Outdoor: likes hiking and sleeping in tents
INFO - Persona Summary for user_13: 
User user_13 preferences: || Career Development - Career: prefers_path industry | Education: studied_at Yale | Skill: has_skill teaching || Content Consumption - Book: likes reading philosophy | Game: plays arcade games | Movie: likes watching fantasy movies | Music: likes indie music || Lifestyle Optimization - 

In [4]:
result

[{'username': 'user_32',
  'full_name': 'Indigo Underwood',
  'age': 39,
  'gender': 'female',
  'score': 0.85,
  'reasoning': '```json\n{\n  "score": 0.85,\n  "reasoning": "User has a hobby in camping, indicating a strong interest in outdoor activities, which aligns well with the classification criteria."\n}\n``` <|end|>'},
 {'username': 'user_39',
  'full_name': 'Finley Nash',
  'age': 23,
  'gender': 'male',
  'score': 0.0,
  'reasoning': '```json\n{\n  "score": 0.0,\n  "reasoning": "No topics in the user persona relate to camping and outdoor activities."\n}\n``` <|end|>'},
 {'username': 'user_08',
  'full_name': 'Hannah Adams',
  'age': 24,
  'gender': 'female',
  'score': 0.7,
  'reasoning': '```json\n{\n  "score": 0.7,\n  "reasoning": "User has a hobby in mountain biking, which is related to outdoor activities. However, no direct interest in camping or broader outdoor activities is mentioned."\n}\n``` <|end|>'},
 {'username': 'user_20',
  'full_name': 'Quinn Vega',
  'age': 19,
 